<div class="alert alert-block alert-success" style="text-align:center;">
Formation Data Scientist - Projet n°7 - OpenClassrooms - Christoph Pruvost - avril/mai 2022
    
<center><a href="https://openclassrooms.com/fr/paths/164/projects/632/assignment">https://openclassrooms.com/fr/paths/164/projects/632/assignment</a></center>
<!-- <p></p>
<ul class="last simple">
<li></li> -->
</div>

# Implémentez un modèle de scoring
<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-size:25px;font-weight:bold;text-align:center">Prétraitements</p>
<p class="last">&nbsp;
<p>Nous sommes Data Scientist au sein d'une société financière, nommée <strong>"Prêt à dépenser"</strong>, &nbsp;qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.</p>

<p>L’entreprise souhaite <strong>mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité </strong>qu’un client rembourse son crédit, puis classifie la demande en crédit accordé ou refusé. Elle souhaite donc développer un <strong>algorithme de classification</strong> en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).</p>
<p>De plus, les chargés de relation client ont fait remonter le fait que les clients sont de plus en plus demandeurs de <strong>transparence</strong> vis-à-vis des décisions d’octroi de crédit. Cette demande de transparence des clients va tout à fait dans le sens des valeurs que l’entreprise veut incarner.</p>
<p><strong>Prêt à dépenser </strong>décide donc de <strong>développer un dashboard interactif</strong> pour que les chargés de relation client puissent à la fois expliquer de façon la plus transparente possible les décisions d’octroi de crédit, mais également permettre à leurs clients de disposer de leurs informations personnelles et de les explorer facilement.</p>
<br><u><b>Les données</b></u>
<p><a href="https://www.kaggle.com/c/home-credit-default-risk/data">Voici les données</a> dont nous aurons besoin pour réaliser le dashboard. Pour plus de simplicité, nous pouvons également les télécharger à <a href="https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip">cette adresse</a>.</p>
<br><u><b>Notre mission</b></u>
<ol>
<li>Construire un modèle de scoring qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.</li>
<li>Construire un dashboard interactif à destination des gestionnaires de la relation client permettant d'interpréter les prédictions faites par le modèle, et d’améliorer la connaissance client des chargés de relation client.</li>
</ol>
<p><strong>Michaël</strong>, notre manager, nous incite à sélectionner un kernel Kaggle pour nous faciliter la préparation des données nécessaires à l’élaboration du modèle de scoring. Nous analyserons ce kernel et l’adapterons pour nous assurer qu’il répond aux besoins de votre mission. Nous pourrons ainsi nous focaliser sur l’élaboration du modèle, son optimisation et sa compréhension.</p>
&nbsp;
<p>Nous allons réaliser ici les <strong>prétraitements des données</strong>.&nbsp;
</p>
</div>

*Le prétraitement s'appuie sur le kernel Kaggle suivant :* https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features/script


## Imports et variables
Importons les librairies nécessaires à notre analyse et définissons quelques variables globales.

In [1]:
import numpy as np
import pandas as pd
import time, datetime, sys, os, gc, re
from contextlib import contextmanager

In [2]:
# Versions des librairies utilisées :
print('Version des librairies utilisées le', datetime.datetime.now(), '\n',
       'Python :', sys.version, '\n',
       'NumPy :',  np.version.full_version, '\n',
       'Pandas :', pd.__version__, '\n',
      )

Version des librairies utilisées le 2022-06-06 09:13:20.271157 
 Python : 3.10.4 | packaged by conda-forge | (main, Mar 24 2022, 17:38:57) [GCC 10.3.0] 
 NumPy : 1.22.3 
 Pandas : 1.4.2 



In [3]:
# Appliquons certaines options globales Pandas et seaborn :
pandas_options = {
    'display.max_rows': 400,
    'display.max_column': 50,
    'display.width': 100,
    'display.max_colwidth': 2000,  # 200
    'display.float_format': '{:,.2f}'.format # arrondi à 2 chiffres après la virgule
}

for cle, valeur in pandas_options.items():
    pd.set_option(cle, valeur)

In [4]:
# Variables
data_dir = '.data'
fichier_donnees_pretraitees = "donnees_pretraitees.ftr" # Pour la sauvegarde à venir.
# debug = True # réduit le nombre d'individus sélectionnés dans le jeu de données.
debug = False

## Définition de fonctions

### Timer
Définissions une fonction qui va nous permettre de mesurer le temps pris par l'exécution des différentes parties du code principal.

In [5]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - Fait en {:.0f}s".format(title, time.time() - t0))

### One-Hot-Encoding
Fonction de One-Hot-Encoding des variables qualitatives, nécessaire au pré-traitement des données.

In [6]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

### Pré-traitements

#### Données d'entraînement et de test
1. Chargement et concaténation des données d'entraînement et de test,
1. Suppression de 4 valeurs aberrantes,
1. Encodage manuel des variables qualitatives avec 2 modes (→ 0 et 1),
1. Encodage One-Hot des des variables qualitatives restantes,
1. Remplacement de valeurs aberrantes de `DAYS_EMPLOYED` (365.243 → NaN),
1. Feature Engineering :
 1. `DAYS_EMPLOYED_PERC` : nombre de jours passés dans l'emploi actuel par rapport à l'age,
 1. `INCOME_CREDIT_PERC` : montant total des revenus par rapport au montant du prêt,
 1. `INCOME_PER_PERSON` : montant total des revenus par rapport au nombre de personnes dans le foyer,
 1. `ANNUITY_INCOME_PERC` : montant des annuités divisé par le montant total de revenu,
 1. `PAYMENT_RATE` : montant des annuité divisé par le montant du crédit.

In [7]:
# Pré-traitement des données d'entraînement et de test: application_train.csv et application_test.csv

def application_train_test(num_rows = None, nan_as_category = False):
    # Ouverture des fichiers de données et concaténation
    df = pd.read_csv(os.path.join(data_dir, 'application_train.csv'), nrows= num_rows)
    test_df = pd.read_csv(os.path.join(data_dir, 'application_test.csv'), nrows= num_rows)
    print("Nb données d'entraînement : {}, Nb données de test : {}".format(len(df), len(test_df)))
    df = pd.concat([df, test_df], ignore_index=True)
    # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
    df = df[df['CODE_GENDER'] != 'XNA']
    
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    # Categorical features with One-Hot encode
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    del test_df
    gc.collect()
    return df

#### bureau.csv et bureau_balance.csv
1. Chargement des fichiers `bureau.csv` et `bureau_balance.csv`,
1. Encodage One-Hot des variables qualitatives des deux fichiers,
1. Feature Engineering :
 1. Création de variables pour bureau_balance.csv : 
    1. min, max, size pour `MONTHS_BALANCE`,
    1. mean pour toutes les colonnes créées par encodage One-Hot,
    1. ajout des variables aux données de `bureau.csv`,
 1. Création de variables pour le jeu de données aggrégées,
    1. Nouvelles variables pour les données qualitatives,
    1. Nouvelles variables pour les données quantitatives,
       1. pour les crédits en cours,
       1. pour les crédits passés,
1. Suppression de la variable `SK_ID_BUREAU` devenue inutile après l'aggrégation,

In [8]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = pd.read_csv(os.path.join(data_dir, 'bureau.csv'), nrows = num_rows)
    bb = pd.read_csv(os.path.join(data_dir, 'bureau_balance.csv'), nrows = num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

#### previous_applications.csv
1. Chargement du fichier `previous_application.csv`,
1. Encodage One-Hot des données qualitatives,
1. Remplacement pour plusieurs variables des valeurs aberrantes du nombre de jours (365 243 → NaN),
1. Feature Engineering :
 1. montant demandé par rapport au montant reçu,
 1. min, max, moyenne des variables quantitatives,
     1. pour l'ensemble des demandes de prêt,
     1. pour les prêts accordés uniquements,
     1. pour les prêts refusés uniquement,
 1. moyenne des variables qualitatives

In [9]:
# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = pd.read_csv(os.path.join(data_dir, 'previous_application.csv'), nrows = num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

### previous_applications.csv
1. Chargement du fichier `POS_CASH_balance.csv`,
1. Encodage One-Hot des variables qualitatives,
1. Feature Engineering :
 1. Variables quantitatives : moyenne, max et taille de certaines variables,
 1. Variables qualitatives : moyenne,
 1. Taille d'un point de vente donné en nombre de dossiers,

In [10]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = pd.read_csv(os.path.join(data_dir, 'POS_CASH_balance.csv'), nrows = num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

### installments_payments.csv
1. Chargement du fichier `installments_payments.csv`,
1. Encodage One-Hot des données qualitatives,
1. Feature Engineering
 1. Variables quantitatives 
     1. Montant payé par le client par rapport au montant de la traite,
     1. Différence entre la traite et le montant payé par le client,
     1. Nombre de jours de dépassement du remboursement du dernier prêt, le cas échéant,
     1. Nombre de jours restants avant rembrousement du dernier prêt, le cas échéant,
     1. Max, moyenne, somme, variance, nombre, pour différentes variables selon les cas,
 1. Variables qualitatives 
     1. Moyenne
 1. Nombre de versements

In [11]:
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = pd.read_csv(os.path.join(data_dir, 'installments_payments.csv'), nrows = num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

### credit_card_balance.csv
1. Chargement du fichier `credit_card_balance.csv`
1. Encodage One-Hot des variables qualitatives,
1. Feature Engineering :
 1. Variables quatitatives : min, max, moyenne, somme, variance
 1. Nombre d'opérations par client

In [12]:
# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = pd.read_csv(os.path.join(data_dir, 'credit_card_balance.csv'), nrows = num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

## Chaîne de traitement principale
Nous définissons ici la chaîne de traitement principale qui utilise les fonctions définies précédemment.

In [13]:
def main(debug = False):
    num_rows = 10000 if debug else None
    print('─'*50)
    with timer("Pré-traitement des fichiers application_train.csv et application_test.csv"):
        print("application_train.csv et application_test.csv")
        df = application_train_test(num_rows)
        print("Dimensions du jeu de données :", df.shape)        
    with timer("Pré-traitement des fichiers bureau.csv et bureau_balance.csv"):
        print('─'*50)
        print("bureau.csv et bureau_balance.csv")
        bureau = bureau_and_balance(num_rows)
        print("Dimensions du jeu de données :", bureau.shape)
        print("Intégration des données au jeu de données principal")
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        del bureau
        gc.collect()
    with timer("Pré-traitement du fichier previous_applications.csv"):
        print('─'*50)
        print("previous_applications.csv")
        prev = previous_applications(num_rows)
        print("Dimensions du jeu de données :", prev.shape)
        print("Intégration des données au jeu de données principal")
        df = df.join(prev, how='left', on='SK_ID_CURR')
        del prev
        gc.collect()
    with timer("Pré-traitement du fichier POS-CASH_balance.csv"):
        print('─'*50)
        print("POS-CASH_balance.csv")
        pos = pos_cash(num_rows)
        print("Dimensions du jeu de données résultant :", pos.shape)
        print("Intégration des données au jeu de données principal")
        df = df.join(pos, how='left', on='SK_ID_CURR')
        del pos
        gc.collect()
    with timer("Pré-traitement du fichier installments_payments.csv"):
        print('installments_payments.csv')
        print('─'*50)
        ins = installments_payments(num_rows)
        print("Dimensions du jeu de données résultant :", ins.shape)
        print("Intégration des données au jeu de données principal")
        df = df.join(ins, how='left', on='SK_ID_CURR')
        del ins
        gc.collect()
    with timer("Pré-traitement du fichier credit_card_balance.csv"):
        print('─'*50)
        print('credit_card_balance.csv')
        cc = credit_card_balance(num_rows)
        print("Dimensions du jeu de données résultant :", cc.shape)
        print("Intégration des données au jeu de données principal")
        df = df.join(cc, how='left', on='SK_ID_CURR')
        del cc
        gc.collect()
    return df

In [14]:
with timer("Traitement intégral"):
    df = main()

──────────────────────────────────────────────────
application_train.csv et application_test.csv
Nb données d'entraînement : 307511, Nb données de test : 48744
Dimensions du jeu de données : (356251, 247)
Pré-traitement des fichiers application_train.csv et application_test.csv - Fait en 5s
──────────────────────────────────────────────────
bureau.csv et bureau_balance.csv
Dimensions du jeu de données : (305811, 116)
Intégration des données au jeu de données principal
Pré-traitement des fichiers bureau.csv et bureau_balance.csv - Fait en 23s
──────────────────────────────────────────────────
previous_applications.csv
Dimensions du jeu de données : (338857, 249)
Intégration des données au jeu de données principal
Pré-traitement du fichier previous_applications.csv - Fait en 28s
──────────────────────────────────────────────────
POS-CASH_balance.csv
Dimensions du jeu de données résultant : (337252, 18)
Intégration des données au jeu de données principal
Pré-traitement du fichier POS-CASH

## Traitement des valeurs aberrantes et manquantes
Nous allons traiter ici les valeurs aberrantes et manquantes restantes après les traitements précédents.

### Valeurs aberrantes

#### Valeurs infinies
Traitons les valeurs infnies du jeu de données en les remplaçant par des NaN.

In [15]:
val_inf = np.isinf(df).sum()
val_inf = val_inf[val_inf > 0]
val_inf

PREV_APP_CREDIT_PERC_MAX        1
REFUSED_APP_CREDIT_PERC_MAX     1
INSTAL_PAYMENT_PERC_MAX        19
dtype: int64

In [16]:
for col in val_inf.index :
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)

### Valeurs manquantes

In [17]:
NAN = df.isna().sum()
NAN[ NAN > 0 ]

TARGET                               48744
AMT_ANNUITY                             36
AMT_GOODS_PRICE                        278
DAYS_EMPLOYED                        64648
OWN_CAR_AGE                         235239
                                     ...  
CC_NAME_CONTRACT_STATUS_nan_MAX     252693
CC_NAME_CONTRACT_STATUS_nan_MEAN    252693
CC_NAME_CONTRACT_STATUS_nan_SUM     252693
CC_NAME_CONTRACT_STATUS_nan_VAR     253385
CC_COUNT                            252693
Length: 617, dtype: int64

### Valeurs manquantes

#### Suppression des variables avec plus de 70% de NaN

In [18]:
# Copie de sauvegarde au cas où
df_backup = df.copy()
df.shape

(356251, 797)

In [19]:
# Suppression des colonnes avec plus de 70% de NaN 
df.dropna(thresh=0.7*df.shape[0], axis=1, inplace=True)

In [20]:
df.shape

(356251, 544)

In [21]:
# Nombre de variables supprimées :
798-545

253

In [22]:
df["TARGET"].head(1)

0   1.00
Name: TARGET, dtype: float64

#### Imputations
Sélectionnons toutes les variables avec des valeurs manquantes, en dehors de la variable cible et procédons à une imputation par la médiane.

In [23]:
var_imputables = [var for var in df.columns if var != 'TARGET' and df[var].isna().any()]
len(var_imputables)

363

In [24]:
# Imputation par la médiane
for variable in var_imputables:
    df[variable].fillna(value=df[variable].median(), inplace=True)

In [25]:
NAN = df.isna().sum()
NAN[ NAN > 0 ]

TARGET    48744
dtype: int64

### Renommage des variables
Certains caractères utilisés pour les noms de variables posent problème lors de l'utilisation de LightGBM. Nous allons donc renommer les variables pour supprimer les caractères problématiques.

In [26]:
# Renommage des colonnes pour corriger un problème avec lightgbm :
# lightgbm.basic.LightGBMError: Do not support special JSON characters in feature name.
# https://stackoverflow.com/questions/60582050/lightgbmerror-do-not-support-special-json-characters-in-feature-name-the-same#62364946
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_ ]+', '', x))

## Sauvegarde des données
Sauvegardons le résultat du pré-traitement afin de pouvoir y accéder pour la modélisation. Les données sauvegardées contiennent à la fois les données d'entraînement et les données de test.

In [27]:
# Sauvegarde du jeu de données intégral (données d'entraînement et de test kaggle)
df.reset_index(drop=True).to_feather(os.path.join(data_dir, fichier_donnees_pretraitees))